### Run the cell below and update the Mongo Host and Port Info

In [1]:
! bash ~/PySyft/scripts/dev_tools.sh

-----------------------------
Containers | Exposed Ports
-----------------------------
"docker inspect" requires at least 1 argument.
See 'docker inspect --help'.

Usage:  docker inspect [OPTIONS] NAME|ID [NAME|ID...]

Return low-level information on Docker objects


In [2]:
## Start here.

In [1]:
# Set your public ports for mongo here.

class EnvSettings:
    MONGO_HOST = "localhost"  # # change to 'localhost'
    MONGO_PORT = "49156"  # change to public port
    MONGO_USERNAME = "root"
    MONGO_PASSWORD = "example"  # nosec
    
env_settings = EnvSettings()

In [2]:
import syft as sy
from syft.core.node.new.user import UserCreate, User, UserUpdate
from syft.core.node.new.mongo_document_store import MongoCollection, MongoDocumentStore, MongoBsonObject
from syft.core.node.new.document_store import StoreClientConfig
from syft.core.node.common.node_table.syft_object import SyftObjectRegistry
from syft.core.node.worker import Worker
from syft.core.node.new.user_stash import UserStash
from syft.core.common.uid import UID
from syft.core.node.new.document_store import UIDPartitionKey

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/shubham/PySyft


In [3]:
mongo_client_config = StoreClientConfig(
    hostname=env_settings.MONGO_HOST,
    port=env_settings.MONGO_PORT,
    username=env_settings.MONGO_USERNAME,
    password=env_settings.MONGO_PASSWORD,
)

In [18]:
store = MongoDocumentStore(client_config=mongo_client_config)

In [19]:
mongo_store_ser = sy.serialize(store, to_bytes=True)

In [20]:
store = sy.deserialize(mongo_store_ser, from_bytes=True)

In [16]:
user_stash = UserStash(store=store)

Using from cache
{-370647245722421374: MongoClient(host=['localhost:49156'], document_class=dict, tz_aware=False, connect=True, tls=False, uuidrepresentation=4)}


In [27]:
user_stash.partition.collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'User_index_name': {'v': 2,
  'key': [('email', 1), ('signing_key', 1), ('verify_key', 1)],
  'unique': True}}

In [28]:
new_user = UserCreate(
    name="Shubham", email="alice1@openmined.org", password="pass", password_verify="pass"
)

In [29]:
user = new_user.to(User)

In [30]:
bson_dict = user.to(MongoBsonObject)

In [31]:
bson_dict.to(UserCreate)

```python
class User:
  id: str = b4e9c96ac5174f32a06b3107abe675fb
  email: str = "alice1@openmined.org"
  name: str = "Shubham"
  hashed_password: str = "$2b$12$qGI8cDL2iCp9bAAm8p8/T.2.IynfSNKMGZtBI5U5avRJvMTNr9gvy"
  salt: str = "$2b$12$qGI8cDL2iCp9bAAm8p8/T."
  signing_key: str = bb84a4a521232bcfae1d93de18c3f932d193092d9022fd5b3db3997c246cc86f
  verify_key: str = 9ee3d04e55384929e3fe3ce6a123b805ad40a0db780e3439ca0abc006c9b523f
  role: str = ServiceRole.GUEST
  institution: str = None
  website: str = None
  created_at: str = None

```

In [32]:
response = user_stash.set(user)

In [33]:
response

Ok(syft.core.node.new.user.User)

In [15]:
# #### Create indexes
# from pymongo import IndexModel, ASCENDING, DESCENDING
# user_stash.partition.db_collection.create_index([
#     ("email", ASCENDING),
#     ("verify_key", ASCENDING)
# ], unique=True)

In [34]:
search_result = user_stash.get_by_email(email="alice1@openmined.org")

In [35]:
search_result.ok() if search_result.is_ok() else search_result.err()

```python
class User:
  id: str = b4e9c96ac5174f32a06b3107abe675fb
  email: str = "alice1@openmined.org"
  name: str = "Shubham"
  hashed_password: str = "$2b$12$qGI8cDL2iCp9bAAm8p8/T.2.IynfSNKMGZtBI5U5avRJvMTNr9gvy"
  salt: str = "$2b$12$qGI8cDL2iCp9bAAm8p8/T."
  signing_key: str = bb84a4a521232bcfae1d93de18c3f932d193092d9022fd5b3db3997c246cc86f
  verify_key: str = 9ee3d04e55384929e3fe3ce6a123b805ad40a0db780e3439ca0abc006c9b523f
  role: str = ServiceRole.GUEST
  institution: str = None
  website: str = None
  created_at: str = None

```

In [36]:
uid_to_update = UID.from_string("b4e9c96ac5174f32a06b3107abe675fb")

In [37]:
update_user = UserUpdate(id=uid_to_update, email="alice1@openmined.org", name="MyBob2", institution="OpenMined")

In [38]:
update_result = user_stash.update(user=update_user.to(User))

In [39]:
update_result.ok() if update_result.is_ok() else update_result.err()

```python
class User:
  id: str = b4e9c96ac5174f32a06b3107abe675fb
  email: str = "alice1@openmined.org"
  name: str = "MyBob2"
  hashed_password: str = None
  salt: str = None
  signing_key: str = None
  verify_key: str = None
  role: str = None
  institution: str = "OpenMined"
  website: str = None
  created_at: str = None

```

In [40]:
user_stash.get_by_email(email="bob@alice.com")

Ok(None)

In [41]:
uid_to_delete = UID.from_string("b4e9c96ac5174f32a06b3107abe675fb")
response = user_stash.delete_by_uid(uid=uid_to_delete)

In [42]:
response

Ok(<class 'syft.core.node.new.response.SyftSuccess'>: ID: b4e9c96ac5174f32a06b3107abe675fb deleted)

### Testing the services

In [43]:
import syft as sy
from syft.core.node.new.user import UserSearch, UserCreate
from syft.core.node.worker import Worker

In [44]:
from syft.core.node.new.mongo_document_store import MongoStoreConfig

In [45]:
worker = Worker(store_config=MongoStoreConfig(client_config=mongo_client_config))

Using from cache
{-370647245722421374: MongoClient(host=['localhost:49156'], document_class=dict, tz_aware=False, connect=True, tls=False, uuidrepresentation=4)}
Using from cache
{-370647245722421374: MongoClient(host=['localhost:49156'], document_class=dict, tz_aware=False, connect=True, tls=False, uuidrepresentation=4)}
Starting Worker: Exciting Lecun - e8eb23ca5abc48f2bea0774ebfdd3c42 [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>]


In [46]:
worker.document_store

In [47]:
client = sy.new_login(node=worker, email="info@openmined.org", password="changethis", port=8081)

Logged into Exciting Lecun as <info@openmined.org>


In [48]:
u_create = UserCreate(name="Shubham", email="shubh@openmined.org", password="pass", password_verify="pass")

In [49]:
client.api.services.user.create(user_create=u_create)

```python
class UserView:
  id: str = 6ddc3cad726341a38a214426869707fa
  email: str = "shubh@openmined.org"
  name: str = "Shubham"
  role: str = ServiceRole.GUEST
  password: str = None
  password_verify: str = None
  verify_key: str = None
  institution: str = None
  website: str = None

```

In [50]:
client.api.services.user.search(name="Shubham")

,type
0,syft.core.node.new.user.UserView


In [51]:
uid_to_delete = UID.from_string("6ddc3cad726341a38a214426869707fa")
client.api.services.user.delete(uid_to_delete)

<class 'syft.core.node.new.response.SyftSuccess'>: ID: 6ddc3cad726341a38a214426869707fa deleted

In [52]:
client.api.services.user.search(name="Shubham")

[]